## `gpu.js` in `pyolite`

Note: this was shamelessly lifted from [bollwyvl/GPU.js in jupyterlite.ipynb]https://gist.github.com/bollwyvl/29f0e8011fe58c7af0915583c8e7d45e

`pyolite` runs inside a [WebWorker](https://developer.mozilla.org/en-US/docs/Web/API/Web_Workers_API). While some features don't work there, a lot of really cool ones do.

- `importScripts` can load arbitrary scripts from The Internet, or the local machine
- the browser's GPU is often accessible there, depending on the browser and settings

Combining these two, we can explore [gpu.js](https://github.com/gpujs/gpu.js), a JS library for working with GPUs, with fallback to pure JS.

In [ ]:
# basically `window` or `self`
import js
# for managing types properly
from pyodide.ffi import to_js
# for the pretties
from IPython.display import Markdown
import numpy

## `importScripts`

`js.importScripts` blocks and just bangs things right onto the global scope.

In [38]:
js.importScripts('https://cdn.jsdelivr.net/npm/gpu.js@latest/dist/gpu-browser.js')

## `GPU`

We now have a `GPU` top-level object.

In [ ]:
GPU = js.GPU.GPU

but what can it do?

In [40]:
def gpu_capabilities():
    checks = {
        "isGPUSupported": "GPU is in-fact supported",
        "isKernelMapSupported": "kernel maps are supported",
        "isOffscreenCanvasSupported": "offscreen canvas is supported",
        "isWebGLSupported": "WebGL v1 is supported",
        "isWebGL2Supported": "WebGL v2 is supported",
        "isHeadlessGLSupported": "headlessgl is supported",
        "isCanvasSupported": "canvas is supported",
        "isGPUHTMLImageArraySupported": "the platform supports HTMLImageArray's",
        "isSinglePrecisionSupported": "the system supports single precision float 32 values",
    }
    results = [
        "### My Browser's `GPU` capabilities",
        f"> agent string `{js.navigator.userAgent}`" 
    ]
    for check, message in checks.items():
        x = "x" if getattr(GPU, check) else " "
        results += [f"- [{x}] {message}"]
    return Markdown("\n".join(results))

gpu_capabilities()

### My Browser's `GPU` capabilities
> agent string `Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36`
- [x] GPU is in-fact supported
- [x] kernel maps are supported
- [x] offscreen canvas is supported
- [x] WebGL v1 is supported
- [x] WebGL v2 is supported
- [ ] headlessgl is supported
- [ ] canvas is supported
- [x] the platform supports HTMLImageArray's
- [x] the system supports single precision float 32 values

## Making a `GPU` instance

First, we need a `GPU`.

In [ ]:
gpu = GPU.new()

## Making a Kernel

Next, we need a _GPU kernel_ (not to be confused with a Jupyter(Lite) kernel!). 

### Defining the kernel function

For starters, we'll just use good-ol `eval` to get a function, and hand that to the kernel. This will follow one of [`gpu.js`'s basic tests](https://github.com/gpujs/gpu.js/blob/develop/test/features/basic-math.js).

In [42]:
kernel_function = js.eval("""function kernelFn(a, b) {
    let sum = 0;
    sum += a[this.thread.y][0] * b[0][this.thread.x];
    sum += a[this.thread.y][1] * b[1][this.thread.x];
    sum += a[this.thread.y][2] * b[2][this.thread.x];
    return sum;
}; kernelFn;
""")
kernel_function

function kernelFn(a, b) {
    let sum = 0;
    sum += a[this.thread.y][0] * b[0][this.thread.x];
    sum += a[this.thread.y][1] * b[1][this.thread.x];
    sum += a[this.thread.y][2] * b[2][this.thread.x];
    return sum;
}

### Defiing the options

Kernels take a number of options, but the required one is `output`, the shape of the output array. We condition the whole thing with `to_js`.

In [43]:
output = to_js([3, 3])

### Instantiate the kernel

In [44]:
kernel = gpu.createKernel(kernel_function).setOutput(output)

## Using the kernel

### Defining the inputs

These _also_ need to be properly conditioned with `to_js`.

In [45]:
a = to_js([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
b = to_js([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

### Calling the kernel

In [46]:
res = kernel(a, b)
res

30,36,42,66,81,96,102,126,150

### Conditioning the output

While it looks flat, there's actually some structure in there: it needs a little [conditioning](https://pyodide.org/en/stable/usage/type-conversions.html#using-javascript-typed-arrays-from-python) to be useful.

In [47]:
arr = numpy.asarray(res.to_py())
arr

array([[ 30.,  36.,  42.],
       [ 66.,  81.,  96.],
       [102., 126., 150.]], dtype=float32)

## And that's "it"

This is just a quick look at using `importScripts` and `GPU.js`. There's loads more to do:

- avoid calling out to a CDN
- benchmarking
- packaging this up as something useful